# Practica 4: Word sense disambiguation

## Downloads

In [121]:
# import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('word2vec_sample')

## NLTK
1)	 Implementar,	 usando	 NLTK	 y	 Python,	 el	 algoritmo	 de	 Lesk simplificado	 para	
desambiguar	el	 sentido	 de	las	 palabras	 (WSD).	 La	 función	 recibirá	 una	 palabra	 y	
una	 frase	que	la	contenga	y	decidirá	el	mejor	sentido	para	esa	palabra.	Las	 frases	
serán	en	inglés	 y	 se	 deberá	eliminar	de	la	 frase, de	la	glosa	 y de	los ejemplos	 de	
cada	sentido	las	‘stopwords’.

In [133]:

from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords as sw
sw_english=sw.words('english')

In [132]:
def lesk_similarity(word:str, sentence:str):
    best = None
    max_ov=0
    ps=[wn.morphy(w) for w in sentence.lower().split() if w not in sw_english]
    
    for s in wn.synsets(word):
        aux=list(set(s.examples()))
        aux.append(s.definition())
        aux=[wn.morphy(w)for sentence in aux for w in sentence.split() ]
        ov=len([word for word in ps if word in aux])
        if max_ov<ov:
            max_ov=ov
            best=s
    return best
        

In [134]:
l=lesk_similarity("bank","“Yesterday I went to the	bank to withdraw the money and the	credit	card did	not	work")
l

Synset('depository_financial_institution.n.01')

In [135]:
l.definition()

'a financial institution that accepts deposits and channels the money into lending activities'

## Embedding similarity
2)	Implementar	un	algoritmo	similar	para	la	desambiguación	semántica	utilizando	
Word	Embeddings	y	una	distancia	de	similitud	semántica	como	la	distancia	coseno


In [100]:
# Word embeddings
import gensim
from nltk.data import find
import numpy as np
from numpy.linalg import norm
# Cargar el modelo de embeding pre-entrenados del NLTK
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [130]:
def embedding_similarity(word:str, sentence: str):
    # Characters to remove
    characters_to_remove = "([\(\)\.\,\‘\“\?\¿\!\¡\;\:\'%\"ª\-])"
    # Create a translation table to remove characters that might pose an issue with the word2vec model
    translation_table = str.maketrans('', '', characters_to_remove)
    best = None
    similarity=-1
    ps=set([w for w in sentence.lower().translate(translation_table).strip().split() if w not in sw_english])
    embedding_s=np.sum([model[w] for w in ps if w in model], axis=0)
    for s in wn.synsets(word):
        aux=list(set(s.examples()))
        aux.append(s.definition())
        aux=set([w for sentence in aux for w in sentence.translate(translation_table).strip().split() if w not in sw_english ])
        embedding_aux=np.sum([model[w] for w in aux if w in model], axis=0)
        cosine = np.dot(embedding_aux,embedding_s)/(norm(embedding_aux)*norm(embedding_s))
        if similarity<cosine:
            similarity=cosine
            best=s
    return best

In [131]:
l=embedding_similarity("bank","Yesterday I went to the	bank to withdraw the money and the	credit	card did	not	work")
l.definition()

'a financial institution that accepts deposits and channels the money into lending activities'